In [25]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import numpy as np
import html
import matplotlib as plt
import re
from itertools import combinations
pd.set_option('display.max_columns', None)

In [26]:
fuzz.ratio("Marmar Restaurant Corp.", "Zara Restaurant Corp.")

91

In [27]:
fuzz.token_set_ratio("La Nueva Amistad Bakery And Restaurant Corp.","El Viejo Jhons Restaurant Corp")

67

In [28]:
df1=pd.read_csv('cleaned_historic_inspections.csv')

In [29]:
df1

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,13610,FD87ABAA-860E-4762-845D-8F0403D0246B,Roadway,Madame Bonte,Beanhouse LLC,Madame Bonte,318,East 84th Street,Manhattan,10028,"318 East 84th Street, Manhattan, NY",50114174,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,yes,1346965.0,TW,no,NaN,yes,06/09/2022 04:26:00 PM,40.776277,-73.952051,8.0,5.0,138.0,1049941.0,1.015460e+09,Yorkville
1,5900,3B07E4C0-07B7-4079-8333-64446CC3EE03,Sidewalk,Seasoned Vegan,"Seasoned Vegan, LLC","Seasoned Vegan, LLC",55,St. Nicholas Avenue,Manhattan,10026,"55 St. Nicholas Avenue, Manhattan, NY",50003337,47.0,3.0,141.0,NaN,NaN,NaN,yes,no,yes,1289526.0,RW,no,NaN,yes,06/26/2020 08:38:00 PM,40.800500,-73.952507,10.0,9.0,216.0,1054995.0,1.018220e+09,Central Harlem South
2,13018,137C575D-DC14-4F9D-83D9-A3FFE513B3B8,Sidewalk,Americas Cafe & Grill,68th Grill Inc.,Americas Cafe & Grill,1159,3rd Avenue,Manhattan,10065,"1159 3rd Avenue, Manhattan, NY",40797684,8.0,8.0,64.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,10/22/2021 11:01:00 AM,40.766845,-73.962708,8.0,4.0,118.0,1043896.0,1.014220e+09,Lenox Hill-Roosevelt Island
3,11630,15270732-2A78-4C24-89DD-BE8DD916F115,Roadway,Sushi Seki,Seki Inc.,Sushi Seki,208,West 23rd Street,Manhattan,10011,"208 West 23rd Street, Manhattan, NY",50005983,NaN,NaN,NaN,44.0,8.0,352.0,no,yes,yes,0.0,OP,no,NaN,yes,12/14/2020 07:54:00 PM,40.744338,-73.996240,4.0,3.0,91.0,1014129.0,1.007720e+09,Hudson Yards-Chelsea-Flatiron-Union Square
4,13137,EF9C8173-91D1-496E-8BD4-B02BEADC2A21,Roadway,Sami & Susu,Amir Nathan,Sami & Susu,190,Orchard Street,Manhattan,10002,"190 Orchard Street, Manhattan, NY",50112624,NaN,NaN,NaN,22.0,8.0,176.0,no,yes,yes,0.0,TW,no,NaN,yes,12/22/2021 12:46:00 PM,40.722124,-73.988160,3.0,1.0,3001.0,1005393.0,1.004120e+09,Chinatown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14423,9799,108C6784-54DB-480B-8627-CF0495D27A0E,Both,King Of Spades Inc.,King Of Spades Inc.,King Of Spades Inc.,1425,College Point Boulevard,Queens,11356,"1425 College Point Boulevard, Queens, NY",50072483,22.0,4.0,88.0,22.0,8.0,176.0,yes,yes,yes,0.0,OP,no,NaN,yes,08/04/2020 03:27:00 PM,40.784739,-73.845776,7.0,19.0,929.0,4098349.0,4.040850e+09,College Point
14424,8215,7B8EC275-D470-4106-819E-03FC0E1F9717,Both,La Queen Tea House Inc.,La Queen Tea House Inc.,La Queen Tea House Inc.,752B,61st Street,Brooklyn,11220,"752B 61st Street, Brooklyn, NY",50105060,26.0,7.0,182.0,26.0,8.0,208.0,yes,yes,no,NaN,NaN,no,NaN,yes,07/13/2020 03:53:00 PM,40.635543,-74.011220,7.0,38.0,118.0,3143885.0,3.057940e+09,Sunset Park East
14425,2895,476F88D2-D3E8-4E83-8686-8BAC2F752CF5,Sidewalk,Aahar Indina Cuisine,Vidhan Bhatt Inc.,Vidhan Bhatt Inc.,10,Murray Street,Manhattan,10007,"10 Murray Street, Manhattan, NY",50049997,25.0,14.0,350.0,NaN,NaN,NaN,yes,no,yes,1298882.0,RW,no,NaN,yes,06/21/2020 01:46:00 PM,40.713298,-74.007773,1.0,1.0,21.0,1001399.0,1.001240e+09,SoHo-TriBeCa-Civic Center-Little Italy
14426,607,FF0D04EF-1D9A-47D6-8F20-8BDBDBCE2792,Sidewalk,Le Cafe Coffee,Le Cafe Coffee LLC,Le Cafe Coffee LLC,145,4th Avenue,Manhattan,10003,"145 4th Avenue, Manhattan, NY",50042698,10.0,10.0,100.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,06/19/2020 01:19:00 PM,40.733916,-73.989872,3.0,2.0,42.0,1077569.0,1.005590e+09,East Village


In [30]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14428 entries, 0 to 14427
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   objectid                                  14428 non-null  int64  
 1   globalid                                  14428 non-null  object 
 2   Seating Interest (Sidewalk/Roadway/Both)  14428 non-null  object 
 3   Restaurant Name                           14428 non-null  object 
 4   Legal Business Name                       14428 non-null  object 
 5   Doing Business As (DBA)                   14427 non-null  object 
 6   Building Number                           14329 non-null  object 
 7   Street                                    14428 non-null  object 
 8   Borough                                   14428 non-null  object 
 9   Postcode                                  14428 non-null  int64  
 10  Business Address                  

In [31]:
df1['Global_Restaurant_ID'] = ''
df1['Latest_Inspection_Row'] = ''

In [33]:
global_restaurant_id_counter = 0

def are_names_similar(name1, name2, threshold=95):
    return fuzz.ratio(name1, name2) > threshold

# Step 3: Create a function to update Global Restaurant ID and Latest Inspection Row
def update_global_id_and_latest_row(group):
    global global_restaurant_id_counter

    # Initialize variables for tracking similar restaurants within the same BIN
    similar_restaurants = {row_index: [row_index] for row_index in group.index}

    # Find similar restaurants within the same BIN
    for (row_index1, row_index2) in combinations(group.index, 2):
        name1 = group.loc[row_index1, 'Restaurant Name']
        name2 = group.loc[row_index2, 'Restaurant Name']

        if are_names_similar(name1, name2):
            similar_restaurants[row_index1].append(row_index2)
            similar_restaurants[row_index2].append(row_index1)

    # Update Global Restaurant ID and Latest Inspection Row for similar restaurants
    for similar_group in similar_restaurants.values():
        if len(similar_group) > 1:
            global_restaurant_id_counter += 1
            group.loc[similar_group, 'Global_Restaurant_ID'] = f"{global_restaurant_id_counter:08d}"

            # Find the row with the latest time of inspection within the similar group
            latest_inspection_row = max(similar_group, key=lambda x: group.loc[x, 'Time of Submission'])
            group.loc[similar_group, 'Latest_Inspection_Row'] = 0
            group.loc[latest_inspection_row, 'Latest_Inspection_Row'] = 1
        else:
            global_restaurant_id_counter += 1
            group.loc[similar_group, 'Global_Restaurant_ID'] = f"{global_restaurant_id_counter:08d}"
            group.loc[similar_group, 'Latest_Inspection_Row'] = 1

    return group

# Step 4: Handle NULL values in the 'BIN' column
df1['BIN'] = df1['BIN'].astype(str).fillna('NULL')

# Step 5: Group by BIN and apply the update function
grouped = df1.groupby('BIN', group_keys=False).apply(update_global_id_and_latest_row)

# Step 6: For remaining rows, update Global Restaurant ID and Latest Inspection Row
remaining_rows = df1[df1['Latest_Inspection_Row'].isna()]
global_restaurant_id_counter += 1
remaining_rows['Global_Restaurant_ID'] = f"{global_restaurant_id_counter:08d}"
remaining_rows['Latest_Inspection_Row'] = 1

# Step 7: Concatenate the updated groups and remaining rows
final_df = pd.concat([grouped, remaining_rows])

# Step 8: Sort the DataFrame if needed
final_df = final_df.sort_values(by=['Global_Restaurant_ID', 'Time of Submission'])

# Display the final DataFrame
final_df

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Global_Restaurant_ID,Latest_Inspection_Row
142,10205,734B44F0-4A16-4BB8-8693-D4DF1963880B,Sidewalk,The Water Hazard,"Konkapot Entertainment, Inc.","Konkapot Entertainment, Inc.",59,Chelsea Piers,Manhattan,10011,"59 Chelsea Piers, Manhattan, NY",50104509,140.0,18.0,2520.0,NaN,NaN,NaN,yes,no,yes,0.0,OP,no,NaN,yes,08/13/2020 01:28:00 PM,40.747999,-74.008496,4.0,3.0,99.0,1000000.0,1.006620e+09,Hudson Yards-Chelsea-Flatiron-Union Square,00000001,1
411,14193,B31D0255-699C-46A8-8A4C-92816BC06D30,Sidewalk,Allora,Rababella Inc.,Allora 5th,292,5th Avenue,Manhattan,10001,"292 5th Avenue, Manhattan, NY",50125184,22.0,12.0,264.0,NaN,NaN,NaN,yes,no,yes,1357871.0,HL,no,NaN,yes,04/25/2023 10:37:00 AM,40.746286,-73.986142,5.0,4.0,76.0,1000000.0,1.008320e+09,Midtown-Midtown South,00000002,1
1961,12180,F23DDE07-39E0-4DF3-88A5-7BA6A918E510,Both,Porchlight,Porchlight West Chelsea LLC,Porchlight,271,11th Avenue,Manhattan,10001,"271 11th Avenue, Manhattan, NY",50046005,57.0,8.0,456.0,62.0,8.0,496.0,yes,yes,yes,0.0,OP,no,NaN,yes,04/05/2021 04:46:00 PM,40.751743,-74.004941,4.0,3.0,99.0,1000000.0,1.006730e+09,Hudson Yards-Chelsea-Flatiron-Union Square,00000004,0
2371,13412,BFE089CE-E96D-4B28-878B-1C224C13F1C5,Both,Porchlight,Porchlight West Chelsea LLC,Ushg,271,11th Avenue,Manhattan,10001,"271 11th Avenue, Manhattan, NY",50046005,76.0,4.0,304.0,50.0,7.0,350.0,yes,yes,yes,0.0,OP,yes,yes,yes,04/18/2022 05:03:00 PM,40.751743,-74.004941,4.0,3.0,99.0,1000000.0,1.006730e+09,Hudson Yards-Chelsea-Flatiron-Union Square,00000004,1
2684,9765,7C0981EC-DF23-45B4-8682-4D0B7077FF13,Both,Le Pain Quotidien,"Apq E65 Ny, LLC",Le Pain Quotidien,861,Lexington Avenue,Manhattan,10028,"861 Lexington Avenue, Manhattan, NY",50105378,40.0,3.0,120.0,40.0,8.0,320.0,yes,yes,no,NaN,NaN,no,NaN,yes,08/03/2020 05:51:00 PM,40.765813,-73.965557,8.0,4.0,120.0,1000000.0,1.013998e+09,Upper East Side-Carnegie Hill,00000005,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14277,3465,F7F1FA77-A43C-471D-8AE3-D1DBF7E16757,Both,Liebman's Delicatessen& Restuarant,Py Deli Bros Inc.,Py Deli Bros Inc.,552,235th,Bronx,10463,"552 235th, Bronx, NY",40986189,60.0,12.0,720.0,60.0,6.0,360.0,yes,yes,yes,1202885.0,RW,no,NaN,yes,06/22/2020 12:19:00 PM,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,00014425,1
14288,1710,D433561D-BD82-4AC9-8133-D6B1B469C271,Both,Henry's End,Bridge 72 LLC,Henry's End,72,72nd Henry Street,Brooklyn,11201,"72 72nd Henry Street, Brooklyn, NY",50089785,20.0,10.0,200.0,6.0,4.0,24.0,yes,yes,yes,1316528.0,OP,yes,yes,yes,06/19/2020 06:21:00 PM,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,00014426,1
14417,972,31F36CEE-4CAF-4B95-81F5-4C70D131F3EC,Both,Kum Sung Bbq,New Kum Sung Inc.,Kum Sung Bbq,4114042,4007th 149th Place,Queens,11354,"4114042 4007th 149th Place, Queens, NY",50071063,18.0,15.0,270.0,18.0,8.0,144.0,yes,yes,yes,1305218.0,RW,no,NaN,yes,06/19/2020 02:22:00 PM,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,00014427,0
9745,9215,5A2F8C27-75A8-4C60-859E-1480D039C78C,Both,Kum Sung Bbq,New Kum Sung Inc.,Kum Sung Bbq,4114042,4007th 149th Place,Queens,11354,"4114042 4007th 149th Place, Queens, NY",50071063,18.0,15.0,270.0,18.0,8.0,144.0,yes,yes,yes,0.0,RW,no,NaN,yes,07/23/2020 03:27:00 PM,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,00014427,1


In [34]:
final_df['Global_Restaurant_ID'].value_counts()

00007182    8
00007217    7
00014006    6
00014068    6
00007950    6
           ..
00005387    1
00005388    1
00005389    1
00005390    1
00014428    1
Name: Global_Restaurant_ID, Length: 12156, dtype: int64

In [35]:
final_df[final_df['Global_Restaurant_ID']=='00007182']

,objectid,globalid,Seating Interest (Sidewalk/Roadway/Both),Restaurant Name,Legal Business Name,Doing Business As (DBA),Building Number,Street,Borough,Postcode,Business Address,Food Service Establishment Permit #,Sidewalk Dimensions (Length),Sidewalk Dimensions (Width),Sidewalk Dimensions (Area),Roadway Dimensions (Length),Roadway Dimensions (Width),Roadway Dimensions (Area),Approved for Sidewalk Seating,Approved for Roadway Seating,Qualify Alcohol,SLA Serial Number,SLA License Type,Landmark District or Building,landmarkDistrict_terms,healthCompliance_terms,Time of Submission,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Global_Restaurant_ID,Latest_Inspection_Row
12022,1084,542D1FD6-0687-4C61-8772-978BD03C2283,Sidewalk,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,10.0,13.0,130.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,06/19/2020 02:57:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,00007182,0
14062,1177,B4AED4FB-4D6F-4534-802C-6FF3E66BB556,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,10.0,8.0,80.0,no,yes,no,NaN,NaN,no,NaN,yes,06/19/2020 03:20:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,00007182,0
5542,10211,16A76904-73DE-4E1B-8A6A-D17EBDFB2C28,Roadway,Love And Dough,Water Street Tavern Corp.,Water Street Tavern Corp.,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,30.0,8.0,240.0,no,yes,no,NaN,NaN,no,NaN,yes,08/13/2020 02:31:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,00007182,0
10213,10212,E3C6154D-2C1B-4246-8246-873B55AF0257,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,30.0,8.0,240.0,no,yes,no,NaN,NaN,no,NaN,yes,08/13/2020 02:34:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,00007182,0
769,10340,C304F754-FA8F-4809-8583-14786BA65329,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,57,Pearl Street,Brooklyn,11201,"57 Pearl Street, Brooklyn, NY",50047483,NaN,NaN,NaN,20.0,8.0,160.0,no,yes,no,NaN,NaN,no,NaN,yes,08/17/2020 04:06:00 PM,40.703056,-73.987582,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,00007182,0
11721,10341,9711D0D3-B678-485A-897F-D510EA20FDEA,Sidewalk,Love And Dough,Water St Tavern Corp.,Love And Dough,57,Pearl Street,Brooklyn,11201,"57 Pearl Street, Brooklyn, NY",50047483,15.0,4.0,60.0,NaN,NaN,NaN,yes,no,no,NaN,NaN,no,NaN,yes,08/17/2020 04:09:00 PM,40.703056,-73.987582,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,00007182,0
12073,12852,D5C7E342-D265-4FE9-8187-B80C6A44DD73,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,20.0,8.0,160.0,no,yes,no,NaN,NaN,no,NaN,yes,09/15/2021 03:01:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,00007182,0
2536,12970,A96E9116-F869-4FEA-8BF1-501535D0912C,Roadway,Love And Dough,Water St Tavern Corp.,Love And Dough,68,Jay Street,Brooklyn,11201,"68 Jay Street, Brooklyn, NY",50047483,NaN,NaN,NaN,40.0,8.0,320.0,no,yes,no,NaN,NaN,no,NaN,yes,10/12/2021 03:33:00 PM,40.702763,-73.986681,2.0,33.0,21.0,3000090.0,3.000400e+09,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,00007182,1


In [ ]:
df2=pd.read_csv('cleaned_current_open_inspections.csv')

In [ ]:
df2['Global_Restaurant_ID'] = ''
df2['Latest_Inspection_Row'] = ''

In [ ]:
df2['BIN'] = df2['BIN'].astype(str).fillna('NULL')

In [ ]:
df2

In [ ]:
def find_matching_name(row, df):
    bin_value = row['BIN']
    restaurant_name = row['RestaurantName']
    
    # Filter df1 for the same BIN and non-null restaurant names
    filtered_df = df[df['BIN'] == bin_value].dropna(subset=['Restaurant Name'])
    
    # Apply fuzzywuzzy to find best match
    matches = filtered_df['Restaurant Name'].apply(lambda x: fuzz.ratio(restaurant_name, x) > 95)
    
    # Get the corresponding Global Restaurant ID from df1
    matching_id = filtered_df.loc[matches, 'Global_Restaurant_ID'].values
    
    # Return the matching Global Restaurant ID
    return matching_id[0] if len(matching_id) > 0 else None

# Apply the function to create a new column in df2
df2['Global_Restaurant_ID'] = df2.apply(lambda row: find_matching_name(row, final_df), axis=1)

In [ ]:
df2

In [ ]:
df2['Global_Restaurant_ID'].info()

In [ ]:
df2[df2['Global_Restaurant_ID'].isna()]

In [ ]:
df3=pd.read_csv('cleaned_dohmh.csv')

In [ ]:
df3['Global_Restaurant_ID'] = ''
df3['Latest_Inspection_Row'] = ''

In [ ]:
df3['BIN'] = df3['BIN'].astype(str).fillna('NULL')

In [ ]:
df3

In [ ]:
def find_matching_name2(row, df):
    bin_value = row['BIN']
    restaurant_DBA = row['DBA']
    street_name = row['STREET']  # Assuming 'StreetName' is the column containing street names
    
    # Filter df for the same BIN and non-null restaurant names
    filtered_df = df[df['BIN'] == bin_value]
    
    # Concatenate restaurant name and street name for fuzzy matching
    filtered_df['Combined_Name'] = filtered_df['Doing Business As (DBA)'] + ' ' + filtered_df['Street']
    
    # Apply fuzzywuzzy to find best match
    matches = filtered_df['Combined_Name'].apply(lambda x: fuzz.ratio(restaurant_DBA + ' ' + street_name, x) > 95)
    
    # Get the corresponding Global Restaurant ID from df
    matching_id = filtered_df.loc[matches, 'Global_Restaurant_ID'].values
    
    # Return the matching Global Restaurant ID
    return matching_id[0] if len(matching_id) > 0 else None

df3['Global_Restaurant_ID'] = df3.apply(lambda row: find_matching_name2(row, final_df), axis=1)

In [ ]:
df3

In [ ]:
df3['Global_Restaurant_ID'].info()

In [ ]:
df3['Global_Restaurant_ID'].value_counts()

In [ ]:
df3[df3['Global_Restaurant_ID']=='00012279']